In [ ]:
%load_ext autotime

# Using Vulkan

This notebook will take you through all the steps in using Vulkan to write policies.\
By the end, you will have:

1. Created a new policy, which can be used for online and batch evaluation,
2. Launched a run of your new policy,
3. Checked its results.

Let's dive in!

In [ ]:
# Used to make our results easier to read
from pprint import pprint

# The actual Vulkan assets used
import vulkan_public.cli.client as vulkan
from vulkan_public.cli.context import Context

In [ ]:
ctx = Context()

## Our First Policy

Let's start by creating a simple policy.\
It takes in a person's (fictional) credit score, and tells us whether that person would be approved or denied.

Later on, we'll see how to get data from an external data source and how to reuse parts of a policy.



### Policy and Policy Versions

In Vulkan, a `Policy` is an umbrella. It has a name, a description and an owner, and can have multiple versions.\
Each `Policy Version` can be used to make decisions separately, and you can have any number of them, making it easy to test and experiment.\
You can think of a Policy as how you make a decision on something, and a Version as each of the ways you'll want to try over time.

In [ ]:
policy_id = vulkan.policy.create_policy(
    ctx,
    name="Credit Policy",
    description="This is our first policy",
)

Each `Policy Version` is defined by its code, the `repository_path` used to create it.\
It is totally independent of others versions, and can have its own definitions and dependencies.

In [ ]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v1",
    repository_path="../examples/policies/simple/",
)

In [ ]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v2",
    repository_path="../examples/policies/simple/",
)

In [ ]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v3",
    repository_path="../examples/policies/simple/",
)

### Runs: How we make decisions

Now that we've created a policy version, **we're ready to make our first decision**.

For that, we'll run our policy directly, giving it some input data.\
In this example, we want to return `APPROVED` if the score is greater than 400, and `DENIED` if not.

The user's data is passed to the `input_data` field. In our example, it's just a "score" value.\
We configure our threshold with the `config_variable` "SCORE_CUTOFF".

In [ ]:
run_id, status = vulkan.run.trigger_run_by_policy_version_id(
    ctx,
    policy_version_id=policy_version_id,
    input_data={"tax_id": "1", "score": 600},
    config_variables={"MINIMUM_SCORE": 400},
    timeout=60,
)

run_data = vulkan.run.get_run_data(ctx, run_id)

# Retrieve the output of our decision.
# In the next section, we'll look into more details for the run data.
pprint(f'Output: {run_data["steps"]["branch"]["output"]}')

#### What happened?

When you trigger a Run using Vulkan, your code will be executed.\
The *actual* execution is happening remotely, on Vulkan's servers.\
It uses the code you uploaded for the `Policy Version`. \
This way, we can handle the infrastructure for you, allowing simple code to scale.

The engine will receive your inputs, the configuration you want to use, and run your code, producing a detailed log of each step.\
Let's take a look at the entire run data object returned.

You'll see that it has a couple of important parts:

1. Run Metadata:
    1.  Run ID: An unique identifier for this run in Vulkan. You can use this to see logs, outputs, or to visualize the decision on the web app.
    2.  Status: Whether or not the run finished successfully.
    3.  Last Updated At: A timestamp for when the run finished.
2. Run Steps:
    1. Each step is identified by the name we gave it, in this case "approved" and "branch";
    2. For each step, we have (1) `output`, which contains the actual output of the step, and (2) `metadata`, which brings information on what kind of step this was, how long it took, etc.

pprint(run_data)


### Marking a version as "Active"

In many cases, we want to have a standard version for a policy.\
This is useful for consumers of this policy: they can just get the current version of "My First Policy", no matter what that is.

In [ ]:
vulkan.policy.set_active_version(
    ctx,
    policy_id=policy_id,
    policy_version_id=policy_version_id,
)

In [ ]:
run_id, status = vulkan.run.trigger_run_by_policy_id(
    ctx,
    policy_id=policy_id,
    input_data={"tax_id": "1", "score": 600},
    timeout=60,
    config_variables={"MINIMUM_SCORE": 750},
)

run_data = vulkan.run.get_run_data(ctx, run_id)
pprint(run_data)

## Components: What, Why and How?

### Creating a couple of Components

In [ ]:
serasa_component_id = vulkan.component.create_component(ctx, name="serasa_component")
serasa_comp_version = vulkan.component.create_component_version(
    ctx,
    component_id=serasa_component_id,
    version_name="v0.0.1",
    repository_path="../examples/components/serasa/",
)

In [ ]:
scr_component_id = vulkan.component.create_component(ctx, name="scr_component")
scr_comp_version = vulkan.component.create_component_version(
    ctx,
    component_id=scr_component_id,
    version_name="v0.0.1",
    repository_path="../examples/components/scr_component/",
)

### Creating a Policy that uses Components

In [ ]:
policy_id = vulkan.policy.create_policy(
    ctx,
    name="Test Policy",
    description="Test Policy Description",
)

In [ ]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v0.0.2",
    repository_path="../examples/policies/onboarding/",
)

In [ ]:
run_id, status = vulkan.run.trigger_run_by_policy_version_id(
    ctx,
    policy_version_id=policy_version_id,
    input_data={"cpf": "1"},
    config_variables={
        "CORTE_SERASA": 500,
        "CORTE_SCR": 600,
        "CORTE_SCR_APROVACAO_AUTOMATICA": 900,
        "CORTE_SCR_RECUSA_AUTOMATICA": 100,
    },
)

run_data = vulkan.run.get_run_data(ctx, run_id)
pprint(run_data)